In [2]:
!pip install transformers datasets 
!pip install rouge.score nltk py7zr
!pip install --upgrade accelerate

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.9 MB/s eta 0:00:00
  Created wheel for rouge.score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=b6fabf350d8fec63f2bd7822135b9dd5b3a1159970edf6dc56e3e48b3235679d
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge.score
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 10.3 MB/s eta 0:00:00
  Attemp

In [3]:
import transformers
from datasets import load_dataset, load_metric, load_from_disk ,DatasetDict
import numpy as np
import nltk
nltk.download('punkt')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
data = load_dataset('samsum',split=["train[:5000]","validation[:625]","test[:625]"])
data = DatasetDict({'train':data[0],'validation':data[1],'test':data[2]})

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
metric = load_metric('rouge')
model_checkpoints = 'facebook/bart-large-cnn'

In [6]:
max_input = 512
max_target = 128
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)
     

def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [dialogue for dialogue in data_to_process['dialogue']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs
     

tokenize_data = data.map(preprocess_data, batched = True, remove_columns=['id', 'dialogue', 'summary'])
    

  0%|          | 0/5 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
#sample the data
train_sample = tokenize_data['train']
validation_sample = tokenize_data['validation']
test_sample = tokenize_data['test']
tokenize_data['train'] = train_sample
tokenize_data['validation'] = validation_sample
tokenize_data['test'] = test_sample
     

In [8]:
#load model
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints,use_fast=False)
     

#collator to create batches. It preprocess data with the given tokenizer
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)
     

#####################
# metrics
# compute rouge for evaluation 
#####################

def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}
     


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [9]:

args = transformers.Seq2SeqTrainingArguments(
    'output_bartSamsum',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    per_device_eval_batch_size= 6,
    gradient_accumulation_steps=3,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=10,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True
    )
#only CUDA available -> fp16=True
trainer = transformers.Seq2SeqTrainer(
    model, 
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['validation'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)
   

In [10]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,0.304804,52.235600,27.208600,43.092800,43.139100,25.427200
2,0.485100,0.319521,52.893800,27.692600,43.595600,43.613200,28.937600
3,0.485100,0.324688,53.420500,27.637100,43.264800,43.300500,30.123200
4,0.190400,0.347889,53.029500,27.558400,42.977800,43.009700,30.361600
5,0.190400,0.384366,52.217900,26.525700,42.589200,42.588700,29.094400
6,0.125000,0.431833,51.723600,26.008300,41.942700,41.974200,32.272000
7,0.125000,0.454412,51.975800,26.090200,41.868000,41.897900,30.179200
8,0.081900,0.489519,51.880400,25.779600,41.860600,41.881700,31.841600
9,0.056800,0.512479,51.506800,25.563300,41.352000,41.346700,31.868800
10,0.056800,0.514933,51.650100,26.032400,41.922200,41.927000,31.673600


TrainOutput(global_step=2780, training_loss=0.17351095213306894, metrics={'train_runtime': 9299.2743, 'train_samples_per_second': 5.377, 'train_steps_per_second': 0.299, 'total_flos': 5.41776150528e+16, 'train_loss': 0.17351095213306894, 'epoch': 10.0})

In [13]:
predict_results = trainer.predict(
            tokenize_data['test'], metric_key_prefix="predict", num_beams=5
        )


In [14]:
predict_results.metrics

{'predict_loss': 0.5358952879905701,
 'predict_rouge1': 50.4775,
 'predict_rouge2': 24.375,
 'predict_rougeL': 40.2427,
 'predict_rougeLsum': 40.2307,
 'predict_gen_len': 30.8224,
 'predict_runtime': 213.4521,
 'predict_samples_per_second': 2.928,
 'predict_steps_per_second': 0.492}

In [15]:
predictions = tokenizer.batch_decode(
                    predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )
import numpy as np
labels = np.where(predict_results.label_ids != -100, predict_results.label_ids, tokenizer.pad_token_id)
actual = tokenizer.batch_decode(
                    labels, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )
from nltk.tokenize import sent_tokenize
preds = [pred.strip() for pred in predictions]
labels = [label.strip() for label in actual]

# rougeLSum expects newline after each sentence
preds = ["\n".join(sent_tokenize(pred)) for pred in preds] #predicted summary
labels = ["\n".join(sent_tokenize(label)) for label in labels] #actual summary



In [16]:
import pandas as pd
df = pd.DataFrame({"Dialogue":data["test"]["dialogue"],"Actual Summary":labels,'Model Summary':preds})
df.to_csv('outputBartSamsum.csv',index=False)

In [17]:
!pip install evaluate rouge_score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.2 MB/s eta 0:00:00


In [18]:
import evaluate
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=preds,references=labels)


In [19]:
results

{'rouge1': 0.4931774503956064,
 'rouge2': 0.23865061177827673,
 'rougeL': 0.394915606678348,
 'rougeLsum': 0.449104808864772}